In [ ]:
!pip install PyDrive

In [1]:
import os
import mykeys

project_name = 'CH15_Evaluation'

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = project_name
os.environ["LANGCHAIN_API_KEY"] = mykeys.get_key('LANG')
os.environ["LANGCHAIN_HUB_API_KEY"] = mykeys.get_key('LANG')
os.environ["OPENAI_API_KEY"] = mykeys.get_key('GPT')
os.environ["GOOGLE_API_KEY"] = mykeys.get_key('GOO')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = mykeys.get_key('HF')
os.environ["UPSTAGE_API_KEY"] = mykeys.get_key('UP')
os.environ["COHERE_API_KEY"] = mykeys.get_key('COH')
os.environ["JINA_API_KEY"] = mykeys.get_key('JINA')
os.environ["ANTHROPIC_API_KEY"] = mykeys.get_key('ANT')
os.environ["DEEPL_API_KEY"] = mykeys.get_key('DEEP')

아래 링크를 복사하여 웹 브라우저에 붙여넣으세요.
https://accounts.google.com/o/oauth2/auth?client_id=35726703810-4v13dfqmilhgv6shlc3cv9i3ktuh73j1.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code
Authentication successful.


In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith(project_name)

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15_Evaluation


In [ ]:
from langchain_teddynote import logging

# set_enable=False 로 지정하면 추적을 하지 않습니다.
logging.langsmith(project_name, set_enable=False)

CH15 평가(Evaluations)

01. 합성 테스트 데이터셋 생성(RAGAS)

In [9]:
!pip uninstall -y ragas
!pip install ragas


Found existing installation: ragas 0.1.21
Uninstalling ragas-0.1.21:
  Successfully uninstalled ragas-0.1.21
  Using cached ragas-0.3.2-py3-none-any.whl.metadata (21 kB)
Using cached ragas-0.3.2-py3-none-any.whl (277 kB)


In [10]:
from langchain_community.document_loaders import PDFPlumberLoader

# 문서 로더 생성
loader = PDFPlumberLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 문서 로딩
docs = loader.load()

# 목차, 끝 페이지 제외
docs = docs[3:-1]

# 문서의 페이지수
len(docs)


19

In [11]:
# metadata 설정(filename 이 존재해야 함)
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]


In [ ]:
from ragas.testset import TestsetGenerator
#from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.transforms.extractors import Keyphrases
#from ragas.testset.docstore import InMemoryDocumentStore

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 데이터셋 생성기
generator_llm = ChatOpenAI(model="gpt-4.1")
# 데이터셋 비평기
critic_llm = ChatOpenAI(model="gpt-4.1")
# 문서 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


ImportError: cannot import name 'Keyphrases' from 'ragas.testset.transforms.extractors' (/opt/anaconda3/envs/LangChain/lib/python3.11/site-packages/ragas/testset/transforms/extractors/__init__.py)

In [15]:
# 텍스트 분할기를 설정합니다.
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# LangChain의 ChatOpenAI 모델을 LangchainLLMWrapper로 감싸 Ragas와 호환되게 만듭니다.
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

# 주요 구문 추출기를 초기화합니다. 위에서 정의한 LLM을 사용합니다.
keyphrase_extractor = KeyphraseExtractor(llm=langchain_llm)

# ragas_embeddings 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# InMemoryDocumentStore를 초기화합니다.
# 이는 문서를 메모리에 저장하고 관리하는 저장소입니다.
docstore = InMemoryDocumentStore(
    splitter=splitter,
    embeddings=ragas_embeddings,
    extractor=keyphrase_extractor,
)


NameError: name 'KeyphraseExtractor' is not defined

In [ ]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    ragas_embeddings,
    docstore=docstore,
)


In [ ]:
# 질문 유형별 분포 결정
# simple: 간단한 질문, reasoning: 추론이 필요한 질문, multi_context: 여러 맥락을 고려해야 하는 질문, conditional: 조건부 질문
distributions = {simple: 0.4, reasoning: 0.2, multi_context: 0.2, conditional: 0.2}


In [ ]:
# 테스트셋 생성
# docs: 문서 데이터, 10: 생성할 질문의 수, distributions: 질문 유형별 분포, with_debugging_logs: 디버깅 로그 출력 여부
testset = generator.generate_with_langchain_docs(
    documents=docs, test_size=10, distributions=distributions, with_debugging_logs=True
)


In [ ]:
# 생성된 테스트셋을 pandas DataFrame으로 변환
test_df = testset.to_pandas()
test_df


In [ ]:
# DataFrame의 상위 5개 행 출력
test_df.head()

# DataFrame을 CSV 파일로 저장
test_df.to_csv("data/ragas_synthetic_dataset.csv", index=False)


https://github.com/teddylee777/langchain-kr/blob/main/16-Evaluations/data/ragas_synthetic_dataset.csv


02. RAGAS 를 활용한 평가

In [24]:
!pip install -Uq faiss-cpu

In [25]:
import pandas as pd

df = pd.read_csv("data/ragas_synthetic_dataset.csv")
df.head()


,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What specific recent developments in generativ...,"['SPRi AI Brief |\n2023-12월호\n삼성전자, 자체 개발 생성 A...",TechRepublic highlighted several recent develo...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
1,What are the dates and location for CES 2024?,['Ⅱ\n. 주요 행사 일정\n행사명 행사 주요 개요\n- 미국 소비자기술 협회(C...,CES 2024 will take place from January 9 to Jan...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
2,What are the key aspects of AI global cooperat...,['문제를 방지하는 조치를 확대\n∙ 형사사법 시스템에서 AI 사용 모범사례를 개발...,The key aspects of AI global cooperation highl...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
3,What measures are suggested to enhance data pr...,['∙ 첨단 AI 시스템의 성능과 한계를 공개하고 적절하거나 부적절한 사용영역을 알...,The context suggests several measures to enhan...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
4,What were the main takeaways from the AI Safet...,"['관련된 경우 해당 국가와 결과를 공유하며, 적절한 시기에 공동 표준 개발을 위해...",The main takeaways from the AI Safety Summit o...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True


In [26]:
from datasets import Dataset

test_dataset = Dataset.from_pandas(df)
test_dataset


Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done'],
    num_rows: 10
})

In [28]:
import ast 
def convert_to_list(example):
    contexts = ast.literal_eval(example["contexts"])
    return {"contexts": contexts}


test_dataset = test_dataset.map(convert_to_list)
print(test_dataset)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done'],
    num_rows: 10
})


In [29]:
test_dataset[1]["contexts"]


['Ⅱ\n. 주요 행사 일정\n행사명 행사 주요 개요\n- 미국 소비자기술 협회(CTA)가 주관하는 세계 최대 가전·IT·소\n비재 전시회로 5G, AR&VR, 디지털헬스, 교통·모빌리티 등\n주요 카테고리 중심으로 기업들이 최신의 기술 제품군을 전시\n- CTA 사피로 회장은 가장 주목받는 섹터로 AI를 조명하였으며,\n모든 산업을 포괄한다는 의미에서 ‘올 온(All on)’을 주제로 한\nCES 2024\n이번 전시에는 500곳 이상의 한국기업 참가 예정\n기간 장소 홈페이지\n2024.1.9~12 미국, 라스베가스 https://www.ces.tech/\n- 머신러닝 및 응용에 관한 국제 컨퍼런스(AIMLA 2024)는\n인공지능 및 머신러닝의 이론, 방법론 및 실용적 접근에 관한\n지식과 최신 연구 결과 공유\n- 이론 및 실무 측면에서 인공지능, 기계학습의 주요 분야를\n논의하고, 학계, 산업계의 연구자와 실무자들에게 해당 분\nAIMLA 2024\n야의 최첨단 개발 소식 공유\n기간 장소 홈페이지\nhttps://ccnet2024.org/aimla\n2024.1.27~28 덴마크, 코펜하겐\n/index\n- AI 발전 협회 컨퍼런스(AAAI)는 AI 연구를 촉진하고, AI 분야\n연구원, 실무자, 과학자, 학생 및 공학자 간 교류의 기회 제공\n- 컨퍼런스에서 AI 관련 기술 발표, 특별 트랙, 초청 연사,\nAAAI Conference\n워크숍, 튜토리얼, 포스터 세션, 주제 발표, 대회, 전시 프\non Artificial\n로그램 등 진행\nIntelligence\n기간 장소 홈페이지\nhttps://aaai.org/aaai-confere\n2024.2.20~27 캐나다, 밴쿠버\nnce/']

In [31]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4.1", temperature=0)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [32]:
batch_dataset = [question for question in test_dataset["question"]]
batch_dataset[:3]


['What specific recent developments in generative AI have been highlighted by TechRepublic in their November 2023 articles?',
 'What are the dates and location for CES 2024?',
 "What are the key aspects of AI global cooperation highlighted in the G7's approach to ethical considerations and regulatory frameworks for advanced AI systems?"]

In [33]:
answer = chain.batch(batch_dataset)
answer[:3]


['According to the provided context, TechRepublic highlighted the rise of on-device AI as a major technology trend in their November 2023 articles. Specifically, they noted the unveiling of Samsung\'s generative AI model "Samsung Gauss," which is designed to operate on devices (on-device AI) and consists of three models: a language model, a code model, and an image model. The on-device operation of Samsung Gauss is emphasized as a key advantage, as it reduces the risk of user information being leaked externally. TechRepublic pointed out that on-device AI is emerging as a significant trend in the technology industry.',
 'CES 2024 will be held from January 9 to January 12, 2024, in Las Vegas, USA.',
 "The G7's approach to ethical considerations and regulatory frameworks for advanced AI systems, as highlighted in the provided context, emphasizes several key aspects of global cooperation:\n\n1. **International Code of Conduct**: The G7 agreed on an International Code of Conduct for Advance

In [35]:
# 'answer' 컬럼 덮어쓰기 또는 추가
if "answer" in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(["answer"]).add_column("answer", answer)
else:
    test_dataset = test_dataset.add_column("answer", answer)


In [36]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=test_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

{'context_precision': 0.6000, 'faithfulness': 0.5068, 'answer_relevancy': 0.9682, 'context_recall': 0.6833}

In [37]:
result_df = result.to_pandas()
result_df.head()


,user_input,retrieved_contexts,response,reference,context_precision,faithfulness,answer_relevancy,context_recall
0,What specific recent developments in generativ...,"[SPRi AI Brief |\n2023-12월호\n삼성전자, 자체 개발 생성 AI...","According to the provided context, TechRepubli...",TechRepublic highlighted several recent develo...,0.5,0.857143,0.919246,0.000000
1,What are the dates and location for CES 2024?,[Ⅱ\n. 주요 행사 일정\n행사명 행사 주요 개요\n- 미국 소비자기술 협회(CT...,CES 2024 will be held from January 9 to Januar...,CES 2024 will take place from January 9 to Jan...,1.0,1.000000,0.957120,1.000000
2,What are the key aspects of AI global cooperat...,[문제를 방지하는 조치를 확대\n∙ 형사사법 시스템에서 AI 사용 모범사례를 개발하...,The G7's approach to ethical considerations an...,The key aspects of AI global cooperation highl...,0.5,0.500000,0.978322,1.000000
3,What measures are suggested to enhance data pr...,[∙ 첨단 AI 시스템의 성능과 한계를 공개하고 적절하거나 부적절한 사용영역을 알리...,The following measures are suggested to enhanc...,The context suggests several measures to enhan...,1.0,0.500000,0.995700,0.500000
4,What were the main takeaways from the AI Safet...,"[관련된 경우 해당 국가와 결과를 공유하며, 적절한 시기에 공동 표준 개발을 위해 ...",The main takeaways from the AI Safety Summit o...,The main takeaways from the AI Safety Summit o...,1.0,0.297297,0.999999,0.666667


In [38]:
result_df.loc[:, "context_precision":"context_recall"]


,context_precision,faithfulness,answer_relevancy,context_recall
0,0.5,0.857143,0.919246,0.000000
1,1.0,1.000000,0.957120,1.000000
2,0.5,0.500000,0.978322,1.000000
3,1.0,0.500000,0.995700,0.500000
4,1.0,0.297297,0.999999,0.666667
5,0.0,0.266667,0.967828,1.000000
6,1.0,0.333333,0.927525,0.666667
7,1.0,0.666667,0.990727,0.500000
8,0.0,0.647059,0.949033,1.000000
9,0.0,0.000000,0.996532,0.500000


03. 생성한 평가용 데이터셋 업로드(HuggingFace Dataset)

In [39]:
import pandas as pd

df = pd.read_csv("data/ragas_synthetic_dataset.csv")
df.head()


,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What specific recent developments in generativ...,"['SPRi AI Brief |\n2023-12월호\n삼성전자, 자체 개발 생성 A...",TechRepublic highlighted several recent develo...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
1,What are the dates and location for CES 2024?,['Ⅱ\n. 주요 행사 일정\n행사명 행사 주요 개요\n- 미국 소비자기술 협회(C...,CES 2024 will take place from January 9 to Jan...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
2,What are the key aspects of AI global cooperat...,['문제를 방지하는 조치를 확대\n∙ 형사사법 시스템에서 AI 사용 모범사례를 개발...,The key aspects of AI global cooperation highl...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
3,What measures are suggested to enhance data pr...,['∙ 첨단 AI 시스템의 성능과 한계를 공개하고 적절하거나 부적절한 사용영역을 알...,The context suggests several measures to enhan...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
4,What were the main takeaways from the AI Safet...,"['관련된 경우 해당 국가와 결과를 공유하며, 적절한 시기에 공동 표준 개발을 위해...",The main takeaways from the AI Safety Summit o...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True


In [40]:
!pip install deepl


In [41]:
import os
from langchain_teddynote.translate import Translator

# api키 설정
deepl_api_key = os.getenv("DEEPL_API_KEY")

# 객체 생성
translator = Translator(deepl_api_key, "EN", "KO")

# 번역 실행
translated_text = translator("hello, nice to meet you")
print(translated_text)


안녕하세요, 만나서 반가워요


In [42]:
from tqdm import tqdm

# 번역
for i, row in tqdm(df.iterrows(), total=len(df), desc="번역 진행 중"):
    df.loc[i, "question_translated"] = translator(row["question"])
    df.loc[i, "ground_truth_translated"] = translator(row["ground_truth"])


번역 진행 중: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.06it/s]


In [43]:
# question, ground_truth 열을 확인합니다.
df.loc[:, ["question", "ground_truth"]].head()


,question,ground_truth
0,What specific recent developments in generativ...,TechRepublic highlighted several recent develo...
1,What are the dates and location for CES 2024?,CES 2024 will take place from January 9 to Jan...
2,What are the key aspects of AI global cooperat...,The key aspects of AI global cooperation highl...
3,What measures are suggested to enhance data pr...,The context suggests several measures to enhan...
4,What were the main takeaways from the AI Safet...,The main takeaways from the AI Safety Summit o...


In [44]:
# question_translated, ground_truth_translated 열을 확인합니다.
df.loc[:, ["question_translated", "ground_truth_translated"]].head()


,question_translated,ground_truth_translated
0,테크리퍼블릭이 2023년 11월 기사에서 강조한 제너레이티브 AI의 최근 발전은 구...,"테크리퍼블릭은 2023년 11월 기사에서 '제너레이티브 AI'의 도입, 라마 2와 ..."
1,CES 2024의 날짜와 장소는 언제인가요?,CES 2024는 2024년 1월 9일부터 1월 12일까지 네바다주 라스베이거스에서...
2,첨단 AI 시스템에 대한 윤리적 고려 사항과 규제 프레임워크에 대한 G7의 접근 방...,G7의 접근 방식에서 강조하는 AI 글로벌 협력의 주요 측면에는 첨단 AI 시스템에...
3,고급 AI 시스템의 맥락에서 데이터 프라이버시를 강화하기 위해 어떤 조치가 제안되나요?,"이 맥락에서는 엄격한 데이터 거버넌스 정책 시행, 데이터 사용의 투명성 보장, 민감..."
4,글로벌 AI 테스트에 관한 AI 안전 서밋의 주요 내용은 무엇인가요?,글로벌 AI 테스트에 관한 AI 안전 서밋의 주요 내용은 세계 지도자들과 최고의 A...


In [45]:
# question, ground_truth 열을 삭제하고 question_translated, ground_truth_translated 열의 이름을 변경합니다.
df.drop(columns=["question", "ground_truth"], inplace=True)
df.rename(
    columns={
        "question_translated": "question",
        "ground_truth_translated": "ground_truth",
    },
    inplace=True,
)
df.head()


,contexts,evolution_type,metadata,episode_done,question,ground_truth
0,"['SPRi AI Brief |\n2023-12월호\n삼성전자, 자체 개발 생성 A...",simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,테크리퍼블릭이 2023년 11월 기사에서 강조한 제너레이티브 AI의 최근 발전은 구...,"테크리퍼블릭은 2023년 11월 기사에서 '제너레이티브 AI'의 도입, 라마 2와 ..."
1,['Ⅱ\n. 주요 행사 일정\n행사명 행사 주요 개요\n- 미국 소비자기술 협회(C...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,CES 2024의 날짜와 장소는 언제인가요?,CES 2024는 2024년 1월 9일부터 1월 12일까지 네바다주 라스베이거스에서...
2,['문제를 방지하는 조치를 확대\n∙ 형사사법 시스템에서 AI 사용 모범사례를 개발...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,첨단 AI 시스템에 대한 윤리적 고려 사항과 규제 프레임워크에 대한 G7의 접근 방...,G7의 접근 방식에서 강조하는 AI 글로벌 협력의 주요 측면에는 첨단 AI 시스템에...
3,['∙ 첨단 AI 시스템의 성능과 한계를 공개하고 적절하거나 부적절한 사용영역을 알...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,고급 AI 시스템의 맥락에서 데이터 프라이버시를 강화하기 위해 어떤 조치가 제안되나요?,"이 맥락에서는 엄격한 데이터 거버넌스 정책 시행, 데이터 사용의 투명성 보장, 민감..."
4,"['관련된 경우 해당 국가와 결과를 공유하며, 적절한 시기에 공동 표준 개발을 위해...",reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,글로벌 AI 테스트에 관한 AI 안전 서밋의 주요 내용은 무엇인가요?,글로벌 AI 테스트에 관한 AI 안전 서밋의 주요 내용은 세계 지도자들과 최고의 A...


In [46]:
# 번역한 데이터셋을 저장합니다.
df.to_csv("data/ragas_synthetic_dataset_translated.csv", index=False)


In [47]:
from datasets import Dataset

# pandas DataFrame을 Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)

# 데이터셋 확인
print(dataset)


Dataset({
    features: ['contexts', 'evolution_type', 'metadata', 'episode_done', 'question', 'ground_truth'],
    num_rows: 10
})


In [48]:
from datasets import Dataset
import os

# pandas DataFrame을 Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)

# 데이터셋 이름 설정 (원하는 이름으로 변경하세요)
dataset_name = "mypsyche98/rag-synthetic-dataset"

# 데이터셋 업로드
dataset.push_to_hub(
    dataset_name,
    private=True,  # private=False로 설정하면 공개 데이터셋이 됩니다.
    split="korean_v1",  # 데이터셋 split 이름 입력
    token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
)


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        : 100%|##########| 31.4kB / 31.4kB            

CommitInfo(commit_url='https://huggingface.co/datasets/mypsyche98/rag-synthetic-dataset/commit/217a4ba65991f839e212af7b7400dcf0b299c0c2', commit_message='Upload dataset', commit_description='', oid='217a4ba65991f839e212af7b7400dcf0b299c0c2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mypsyche98/rag-synthetic-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mypsyche98/rag-synthetic-dataset'), pr_revision=None, pr_num=None)

https://huggingface.co/datasets/mypsyche98/rag-synthetic-dataset

04. LangSmith 데이터셋 생성

In [49]:
import pandas as pd

# 질문과 답변 목록
inputs = [
    "삼성전자가 만든 생성형 AI의 이름은 무엇인가요?",
    "미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행정명령을 발표한 날은 언제인가요?",
    "코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.",
]

# 질문에 대한 답변 목록
outputs = [
    "삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.",
    "2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.",
    "코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스 상태를 추적하고 투명성을 확보하기 위한 플랫폼입니다. 12개 기관과 협력하여 2,000여 개 데이터셋의 출처 정보를 제공하며, 개발자들이 데이터의 구성과 계보를 쉽게 파악할 수 있게 돕습니다.",
]

# 질문과 답변 쌍 생성
qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]

# 데이터프레임으로 변환
df = pd.DataFrame(qa_pairs)

# 데이터프레임 출력
df.head()


,question,answer
0,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.
1,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.
2,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스...


In [63]:
!pip uninstall -y numpy numba streamlit
!pip install numpy numba streamlit

Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6
Found existing installation: numba 0.61.2
Uninstalling numba-0.61.2:
  Successfully uninstalled numba-0.61.2
Found existing installation: streamlit 1.43.2
Uninstalling streamlit-1.43.2:
  Successfully uninstalled streamlit-1.43.2
  Using cached numpy-2.3.2-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached numba-0.61.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.8 kB)
  Using cached numpy-2.2.6-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
Using cached numba-0.61.2-cp311-cp311-macosx_11_0_arm64.whl (2.8 MB)
Using cached numpy-2.2.6-cp311-cp311-macosx_14_0_arm64.whl (5.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 24.5 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [streamlit]━━━━━━━━ 2/3 [streamlit]


In [60]:
!pip install -Uq numba transformers onnx

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 6.32.0 which is incompatible.
langchain-upstage 0.7.3 requires tokenizers<0.21.0,>=0.20.0, but you have tokenizers 0.21.4 which is incompatible.
opentelemetry-proto 1.27.0 requires protobuf<5.0,>=3.19, but you have protobuf 6.32.0 which is incompatible.
streamlit 1.43.2 requires protobuf<6,>=3.20, but you have protobuf 6.32.0 which is incompatible.


In [64]:
import pandas as pd
from datasets import load_dataset, Dataset
import os

# huggingface Dataset에서 repo_id로 데이터셋 다운로드
dataset = load_dataset(
    "mypsyche98/rag-synthetic-dataset",  # 데이터셋 이름
    token=os.environ["HUGGINGFACEHUB_API_TOKEN"],  # private 데이터인 경우 필요합니다.
)

# 데이터셋에서 split 기준으로 조회
huggingface_df = dataset["korean_v1"].to_pandas()
huggingface_df.head()


data/korean_v1-00000-of-00001.parquet:   0%|          | 0.00/31.4k [00:00<?, ?B/s]

Generating korean_v1 split:   0%|          | 0/10 [00:00<?, ? examples/s]

,contexts,evolution_type,metadata,episode_done,question,ground_truth
0,"['SPRi AI Brief |\n2023-12월호\n삼성전자, 자체 개발 생성 A...",simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,테크리퍼블릭이 2023년 11월 기사에서 강조한 제너레이티브 AI의 최근 발전은 구...,"테크리퍼블릭은 2023년 11월 기사에서 '제너레이티브 AI'의 도입, 라마 2와 ..."
1,['Ⅱ\n. 주요 행사 일정\n행사명 행사 주요 개요\n- 미국 소비자기술 협회(C...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,CES 2024의 날짜와 장소는 언제인가요?,CES 2024는 2024년 1월 9일부터 1월 12일까지 네바다주 라스베이거스에서...
2,['문제를 방지하는 조치를 확대\n∙ 형사사법 시스템에서 AI 사용 모범사례를 개발...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,첨단 AI 시스템에 대한 윤리적 고려 사항과 규제 프레임워크에 대한 G7의 접근 방...,G7의 접근 방식에서 강조하는 AI 글로벌 협력의 주요 측면에는 첨단 AI 시스템에...
3,['∙ 첨단 AI 시스템의 성능과 한계를 공개하고 적절하거나 부적절한 사용영역을 알...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,고급 AI 시스템의 맥락에서 데이터 프라이버시를 강화하기 위해 어떤 조치가 제안되나요?,"이 맥락에서는 엄격한 데이터 거버넌스 정책 시행, 데이터 사용의 투명성 보장, 민감..."
4,"['관련된 경우 해당 국가와 결과를 공유하며, 적절한 시기에 공동 표준 개발을 위해...",reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,글로벌 AI 테스트에 관한 AI 안전 서밋의 주요 내용은 무엇인가요?,글로벌 AI 테스트에 관한 AI 안전 서밋의 주요 내용은 세계 지도자들과 최고의 A...


In [65]:
from langsmith import Client

client = Client()
dataset_name = "RAG_EVAL_DATASET"


# 데이터셋 생성 함수
def create_dataset(client, dataset_name, description=None):
    for dataset in client.list_datasets():
        if dataset.name == dataset_name:
            return dataset

    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description=description,
    )
    return dataset


# 데이터셋 생성
dataset = create_dataset(client, dataset_name)

# 생성된 데이터셋에 예제 추가
client.create_examples(
    inputs=[{"question": q} for q in df["question"].tolist()],
    outputs=[{"answer": a} for a in df["answer"].tolist()],
    dataset_id=dataset.id,
)


{'example_ids': ['2e043f96-3f75-4a52-b65b-dbad66f4e58f',
  '2c20a8eb-ea28-4a83-8535-1f602cdcfc57',
  '6c99b3e8-92f9-460f-9861-08699eaf317b'],
 'count': 3}

In [66]:
# 새로운 질문 목록
new_questions = [
    "삼성전자가 만든 생성형 AI의 이름은 무엇인가요?",
    "구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?",
]

# 새로운 답변 목록
new_answers = [
    "삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.",
    "사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 달러를 우선 투자하고 향후 15억 달러를 추가로 투자하기로 했습니다.",
]

# UI에서 업데이트된 버전 확인
client.create_examples(
    inputs=[{"question": q} for q in new_questions],
    outputs=[{"answer": a} for a in new_answers],
    dataset_id=dataset.id,
)


{'example_ids': ['f7d94096-7bcb-439b-832e-96d2dbd27169',
  '19993858-dd49-44a9-b7d3-529a442eb39a'],
 'count': 2}

05. LLM-as-Judge

In [70]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI

# PDFRAG 객체 생성
rag = PDFRAG(
    "data/SPRI_AI_Brief_2023년12월호_F.pdf",
    ChatOpenAI(model="gpt-4.1", temperature=0),
)

# 검색기(retriever) 생성
retriever = rag.create_retriever()

# 체인(chain) 생성
chain = rag.create_chain(retriever)

# 질문에 대한 답변 생성
chain.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")


'삼성전자가 자체 개발한 생성형 AI의 이름은 ‘삼성 가우스’입니다.'

In [71]:
# 질문에 대한 답변하는 함수를 생성
def ask_question(inputs: dict):
    return {"answer": chain.invoke(inputs["question"])}


In [72]:
# 사용자 질문 예시
llm_answer = ask_question(
    {"question": "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?"}
)
llm_answer


{'answer': '삼성전자가 자체 개발한 생성형 AI의 이름은 ‘삼성 가우스’입니다.'}

In [73]:
# evaluator prompt 출력을 위한 함수
def print_evaluator_prompt(evaluator):
    return evaluator.evaluator.prompt.pretty_print()


In [74]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# qa 평가자 생성
qa_evalulator = LangChainStringEvaluator("qa")

# 프롬프트 출력
print_evaluator_prompt(qa_evalulator)


You are a teacher grading a quiz.
You are given a question, the student's answer, and the true answer, and are asked to score the student answer as either CORRECT or INCORRECT.

Example Format:
QUESTION: question here
STUDENT ANSWER: student's answer here
TRUE ANSWER: true answer here
GRADE: CORRECT or INCORRECT here

Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer. It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. Begin!

QUESTION: {query}
STUDENT ANSWER: {result}
TRUE ANSWER: {answer}
GRADE:


In [75]:
dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[qa_evalulator],
    experiment_prefix="RAG_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "QA Evaluator 를 활용한 평가",
    },
)


View the evaluation results for experiment: 'RAG_EVAL-ee6f095d' at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=bd422628-829b-45bf-9010-15ca9e0eb484




0it [00:00, ?it/s]

In [76]:
# Context 를 반환하는 RAG 결과 반환 함수
def context_answer_rag_answer(inputs: dict):
    context = retriever.invoke(inputs["question"])
    return {
        "context": "\n".join([doc.page_content for doc in context]),
        "answer": chain.invoke(inputs["question"]),
        "query": inputs["question"],
    }


In [77]:
# 함수 실행
context_answer_rag_answer(
    {"question": "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?"}
)


{'context': '▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··························································· 10\n   ▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11\n   ▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망··········································· 12\nSPRi AI Brief |  \n2023-12월호\n10\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\nKEY Contents\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, \n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유\n어시스턴트를 적용한 구글 픽셀(Pixel)과 경쟁할 것으로 예상\n☞ 출처 : 삼성전자, ‘삼성 AI 포럼’서 자체 개발 생성형 AI ‘삼성 가우스’ 공개, 2023.11.08.\n삼성전

In [78]:
# cot_qa 평가자 생성
cot_qa_evaluator = LangChainStringEvaluator(
    "cot_qa",
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],  # LLM 이 생성한 답변
        "reference": run.outputs["context"],  # Context
        "input": example.inputs["question"],  # 데이터셋의 질문
    },
)

# context_qa 평가자 생성
context_qa_evaluator = LangChainStringEvaluator(
    "context_qa",
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],  # LLM 이 생성한 답변
        "reference": run.outputs["context"],  # Context
        "input": example.inputs["question"],  # 데이터셋의 질문
    },
)

# evaluator prompt 출력
print_evaluator_prompt(context_qa_evaluator)


You are a teacher grading a quiz.
You are given a question, the context the question is about, and the student's answer. You are asked to score the student's answer as either CORRECT or INCORRECT, based on the context.

Example Format:
QUESTION: question here
CONTEXT: context the question is about here
STUDENT ANSWER: student's answer here
GRADE: CORRECT or INCORRECT here

Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer. It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. Begin!

QUESTION: {query}
CONTEXT: {context}
STUDENT ANSWER: {result}
GRADE:


In [79]:
# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
evaluate(
    context_answer_rag_answer,
    data=dataset_name,
    evaluators=[cot_qa_evaluator, context_qa_evaluator],
    experiment_prefix="RAG_EVAL",
    metadata={
        "variant": "COT_QA & Context_QA Evaluator 를 활용한 평가",
    },
)


View the evaluation results for experiment: 'RAG_EVAL-b8523f30' at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=0fa3c0f2-6b7e-4cd5-92b2-5f86f5e51dbc




0it [00:00, ?it/s]

,inputs.question,outputs.context,outputs.answer,outputs.query,error,reference.answer,feedback.COT Contextual Accuracy,feedback.Contextual Accuracy,execution_time,example_id,id
0,구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,"£ 구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공\nn 구...","아니요, 구글이 20억 달러를 투자한 기업은 테디노트가 아니라 앤스로픽(Anthro...",구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,None,"사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 ...",1,1,2.632281,19993858-dd49-44a9-b7d3-529a442eb39a,c410dffd-9889-4e74-87e6-0fecd18962b2
1,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,"▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··············...",삼성전자가 만든 생성형 AI의 이름은 ‘삼성 가우스’입니다.,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,None,삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.,1,1,1.755885,f7d94096-7bcb-439b-832e-96d2dbd27169,342a8f77-35af-46c3-8f21-813a217ef286
2,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,"▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··············...",삼성전자가 만든 생성형 AI의 이름은 ‘삼성 가우스’입니다.,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,None,삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.,1,1,1.235339,2c20a8eb-ea28-4a83-8535-1f602cdcfc57,089568f5-eec5-4361-811b-e3cae347cc43
3,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,"SPRi AI Brief | \n2023-12월호\n8\n코히어, 데이터 투명성 ...",코히어(Cohere)의 데이터 출처 탐색기(Data Provenance Explor...,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,None,코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스...,1,1,2.535723,2e043f96-3f75-4a52-b65b-dbad66f4e58f,fe1ee6e5-7cef-46b1-8b79-f746d5210df4
4,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,1. 정책/법제 \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미...,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,None,2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.,1,1,1.440293,6c99b3e8-92f9-460f-9861-08699eaf317b,fc796fac-aa1e-4cce-805f-12c3f8acdfd4


In [80]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# 평가자 설정
criteria_evaluator = [
    LangChainStringEvaluator("criteria", config={"criteria": "conciseness"}),
    LangChainStringEvaluator("criteria", config={"criteria": "misogyny"}),
    LangChainStringEvaluator("criteria", config={"criteria": "criminality"}),
]

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=criteria_evaluator,
    experiment_prefix="CRITERIA-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "criteria 를 활용한 평가",
    },
)


View the evaluation results for experiment: 'CRITERIA-EVAL-52c86776' at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=c39e84f4-0d9f-48d9-90c8-e7e5746206c4




0it [00:00, ?it/s]

In [81]:
experiment_results

,inputs.question,outputs.answer,error,reference.answer,feedback.conciseness,feedback.misogyny,feedback.criminality,execution_time,example_id,id
0,구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,"아니요, 구글이 20억 달러를 투자한 기업은 테디노트가 아니라 앤스로픽(Anthro...",None,"사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 ...",0,0,0,2.006998,19993858-dd49-44a9-b7d3-529a442eb39a,b88bb835-ee29-4630-b09f-d4b10831d722
1,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 ‘삼성 가우스’입니다.,None,삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.,1,0,0,1.439164,f7d94096-7bcb-439b-832e-96d2dbd27169,bf451d9b-42b3-4a1f-a500-bb0ba7114551
2,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 ‘삼성 가우스’입니다.,None,삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.,1,0,0,1.117275,2c20a8eb-ea28-4a83-8535-1f602cdcfc57,1a27b78c-907a-4364-bc80-ec49f569d55d
3,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,코히어(Cohere)의 데이터 출처 탐색기(Data Provenance Explor...,None,코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스...,1,0,0,3.105696,2e043f96-3f75-4a52-b65b-dbad66f4e58f,e6117c07-8a0e-4830-8490-5530fb009f57
4,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,None,2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.,1,0,0,2.093883,6c99b3e8-92f9-460f-9861-08699eaf317b,6c7c1ccf-5a15-4b46-a309-8fa15cfedb2b


In [84]:
from langsmith.evaluation import LangChainStringEvaluator
from langchain_openai import ChatOpenAI

# labeled_criteria 평가자 생성
labeled_criteria_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm": ChatOpenAI(temperature=0.0, model="gpt-4.1"),
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": example.outputs["answer"],  # 정답 답변
        "input": example.inputs["question"],
    },
)

# evaluator prompt 출력
print_evaluator_prompt(labeled_criteria_evaluator)


You are assessing a submitted answer on a given task or input based on a set of criteria. Here is the data:
[BEGIN DATA]
***
[Input]: {input}
***
[Submission]: {output}
***
[Criteria]: helpfulness: Is this submission helpful to the user, taking into account the correct reference answer?
***
[Reference]: {reference}
***
[END DATA]
Does the submission meet the Criteria? First, write out in a step by step manner your reasoning about each criterion to be sure that your conclusion is correct. Avoid simply stating the correct answers at the outset. Then print only the single character "Y" or "N" (without quotes or punctuation) on its own line corresponding to the correct answer of whether the submission meets all criteria. At the end, repeat just the letter again by itself on a new line.


In [86]:
from langsmith.evaluation import evaluate

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
experiment_results = evaluate(
    context_answer_rag_answer,
    data=dataset_name,
    evaluators=[labeled_criteria_evaluator, relevance_evaluator],
    experiment_prefix="LABELED-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "labeled_criteria evaluator 활용한 평가",
    },
)
experiment_results

View the evaluation results for experiment: 'LABELED-EVAL-d67fa40e' at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=df6b4700-e332-4892-a30c-68d71fa480d7




0it [00:00, ?it/s]

,inputs.question,outputs.context,outputs.answer,outputs.query,error,reference.answer,feedback.helpfulness,feedback.relevance,execution_time,example_id,id
0,구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,"£ 구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공\nn 구...","아니요, 구글이 20억 달러를 투자한 기업은 테디노트가 아니라 앤스로픽(Anthro...",구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,None,"사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 ...",1,1,3.380211,19993858-dd49-44a9-b7d3-529a442eb39a,5dc385b1-b8ea-48d4-9762-a1e15b6dbff2
1,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,"▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··············...",삼성전자가 만든 생성형 AI의 이름은 '삼성 가우스'입니다.,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,None,삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.,0,1,1.734427,f7d94096-7bcb-439b-832e-96d2dbd27169,2550e50a-6a5c-4536-bcaa-4a11730e6c93
2,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,"▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··············...",삼성전자가 만든 생성형 AI의 이름은 ‘삼성 가우스’입니다.,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,None,삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.,1,1,3.268328,2c20a8eb-ea28-4a83-8535-1f602cdcfc57,2690653d-54d2-4c23-8d55-fdee74cddc5c
3,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,"SPRi AI Brief | \n2023-12월호\n8\n코히어, 데이터 투명성 ...",코히어(Cohere)의 데이터 출처 탐색기(Data Provenance Explor...,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,None,코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스...,1,1,4.529326,2e043f96-3f75-4a52-b65b-dbad66f4e58f,d06f8807-a763-4f70-a9b5-2130db87e382
4,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,1. 정책/법제 \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미...,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,None,2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.,1,1,3.246260,6c99b3e8-92f9-460f-9861-08699eaf317b,68964b75-8b26-4754-b363-cb53e218031a


In [87]:
from langsmith.evaluation import LangChainStringEvaluator

# 점수를 반환하는 평가자 생성
labeled_score_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "criteria": {
            "accuracy": "How accurate is this prediction compared to the reference on a scale of 1-10?"
        },
        "normalize_by": 10,
        "llm": ChatOpenAI(temperature=0.0, model="gpt-4.1"),
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    },
)

print_evaluator_prompt(labeled_score_evaluator)


================================ System Message ================================

You are a helpful assistant.

================================ Human Message =================================

[Instruction]
Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below. {criteria}[Ground truth]
{reference}
Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must rate the response on a scale of 1 to 10 by strictly following this format: "[[rating]]", for example: "Rating: [[5]]".

[Question]
{input}

[The Start of Assistant's Answer]
{prediction}
[The End of Assistant's Answer]


In [88]:
from langsmith.evaluation import evaluate

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[labeled_score_evaluator],
    experiment_prefix="LABELED-SCORE-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "labeled_score 활용한 평가",
    },
)
experiment_results

View the evaluation results for experiment: 'LABELED-SCORE-EVAL-b1351671' at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=705bcb31-f669-4d9f-813a-bfdd1efff514




0it [00:00, ?it/s]

,inputs.question,outputs.answer,error,reference.answer,feedback.score_string:accuracy,execution_time,example_id,id
0,구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,"아니요, 구글이 20억 달러를 투자한 기업은 테디노트가 아니라 앤스로픽(Anthro...",None,"사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 ...",1.0,1.991753,19993858-dd49-44a9-b7d3-529a442eb39a,02c48d46-6b47-414a-8da7-7851f83f2389
1,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 ‘삼성 가우스’입니다.,None,삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.,0.2,1.075399,f7d94096-7bcb-439b-832e-96d2dbd27169,2a0b9f9a-e098-455c-9f4e-ca18fa96f800
2,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 ‘삼성 가우스’입니다.,None,삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.,1.0,0.898308,2c20a8eb-ea28-4a83-8535-1f602cdcfc57,9addcd6d-fcf1-499d-89aa-6ebb7565f553
3,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,코히어(Cohere)의 데이터 출처 탐색기(Data Provenance Explor...,None,코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스...,1.0,11.239015,2e043f96-3f75-4a52-b65b-dbad66f4e58f,5ab915e2-f0ed-4bf7-b595-91dd7f43dfdf
4,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,None,2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.,1.0,1.121556,6c99b3e8-92f9-460f-9861-08699eaf317b,c0c73c10-41b0-47a4-ba3b-20f1242847b1


06. 임베딩 기반 평가(embedding_distance)

In [89]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI

# PDFRAG 객체 생성
rag = PDFRAG(
    "data/SPRI_AI_Brief_2023년12월호_F.pdf",
    ChatOpenAI(model="gpt-4o-mini", temperature=0),
)

# 검색기(retriever) 생성
retriever = rag.create_retriever()

# 체인(chain) 생성
chain = rag.create_chain(retriever)

# 질문에 대한 답변 생성
chain.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")


"삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다."

In [90]:
# 질문에 대한 답변하는 함수를 생성
def ask_question(inputs: dict):
    return {"answer": chain.invoke(inputs["question"])}


In [92]:
from langsmith.evaluation import LangChainStringEvaluator
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_upstage import UpstageEmbeddings
from langchain_openai import OpenAIEmbeddings
import os

# 토크나이저 병렬화 설정(HuggingFace 모델 사용)
os.environ["TOKENIZERS_PARALLELISM"] = "true"

model_name = "BAAI/bge-m3"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "mps"},  # cuda, cpu
    # encode_kwargs={"normalize_embeddings": True},
)

# 임베딩 모델 평가자 생성
hf_embedding_evaluator = LangChainStringEvaluator(
    "embedding_distance",
    config={
        # OpenAIEmbeddings 가 기본 값으로 설정되어 있으나, 변경이 가능
        "embeddings": hf_embeddings,
        "distance_metric": "cosine",  # "cosine", "euclidean", "chebyshev", "hamming", and "manhattan"
    },
)

upstage_embedding_evaluator = LangChainStringEvaluator(
    "embedding_distance",
    config={
        # OpenAIEmbeddings 가 기본 값으로 설정되어 있으나, 변경이 가능
        "embeddings": UpstageEmbeddings(model="solar-embedding-1-large-query"),
        "distance_metric": "euclidean",  # "cosine", "euclidean", "chebyshev", "hamming", and "manhattan"
    },
)

openai_embedding_evaluator = LangChainStringEvaluator(
    "embedding_distance",
    config={
        # OpenAIEmbeddings 가 기본 값으로 설정되어 있으나, 변경이 가능
        "embeddings": OpenAIEmbeddings(model="text-embedding-3-small"),
        "distance_metric": "euclidean",  # "cosine", "euclidean", "chebyshev", "hamming", and "manhattan"
    },
)


In [93]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[
        hf_embedding_evaluator,
        upstage_embedding_evaluator,
        openai_embedding_evaluator,
    ],
    experiment_prefix="EMBEDDING-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "embedding_distance 활용한 평가",
    },
)


View the evaluation results for experiment: 'EMBEDDING-EVAL-27215bb2' at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=ddefd859-af9f-444e-8bd3-827ec5f12f0a




0it [00:00, ?it/s]

In [94]:
experiment_results

,inputs.question,outputs.answer,error,reference.answer,feedback.embedding_cosine_distance,feedback.embedding_euclidean_distance,execution_time,example_id,id
0,구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,구글이 앤스로픽에 최대 20억 달러를 투자하기로 합의한 것이 사실입니다. 테디노트에...,None,"사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 ...",0.146131,0.757220,1.734153,19993858-dd49-44a9-b7d3-529a442eb39a,bb624ec2-a11a-476e-b8f6-3165d6c96d7b
1,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 '삼성 가우스'입니다.,None,삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.,0.230936,0.764774,3.799084,f7d94096-7bcb-439b-832e-96d2dbd27169,01087f10-75b9-4818-8cc1-d657b065853f
2,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 '삼성 가우스'입니다.,None,삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.,0.012339,0.255849,1.572802,2c20a8eb-ea28-4a83-8535-1f602cdcfc57,d8830140-4ef5-4b9d-ab76-3b56e677828f
3,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,코히어는 2023년 10월 25일에 '데이터 출처 탐색기(Data Provenanc...,None,코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스...,0.219000,0.785272,3.858580,2e043f96-3f75-4a52-b65b-dbad66f4e58f,65edf738-d468-42dd-aa1e-6cf1cf396f33
4,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,2023년 10월 30일입니다.,None,2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.,0.283018,1.053067,1.446268,6c99b3e8-92f9-460f-9861-08699eaf317b,8ba5723e-13be-40bb-a636-a2edb8ba6885


07. 사용자 정의(Custom) LLM 평가

In [95]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI

# PDFRAG 객체 생성
rag = PDFRAG(
    "data/SPRI_AI_Brief_2023년12월호_F.pdf",
    ChatOpenAI(model="gpt-4.1", temperature=0),
)

# 검색기(retriever) 생성
retriever = rag.create_retriever()

# 체인(chain) 생성
chain = rag.create_chain(retriever)

# 질문에 대한 답변 생성
chain.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")


'삼성전자가 자체 개발한 생성형 AI의 이름은 ‘삼성 가우스’입니다.'

In [96]:
# 질문에 대한 답변하는 함수를 생성
def ask_question(inputs: dict):
    return {"answer": chain.invoke(inputs["question"])}


In [97]:
from langsmith.schemas import Run, Example
import random


def random_score_evaluator(run: Run, example: Example) -> dict:
    # 랜덤 점수 반환
    score = random.randint(1, 11)
    return {"key": "random_score", "score": score}


In [98]:
from langsmith.evaluation import evaluate

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[random_score_evaluator],
    experiment_prefix="CUSTOM-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "랜덤 점수 평가자",
    },
)
experiment_results

View the evaluation results for experiment: 'CUSTOM-EVAL-fe6a52b0' at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=5eac3d17-d24f-458c-ac8a-4d98523e22f2




0it [00:00, ?it/s]

,inputs.question,outputs.answer,error,reference.answer,feedback.random_score,execution_time,example_id,id
0,구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,"아니요, 구글이 20억 달러를 투자한 기업은 테디노트가 아니라 앤스로픽(Anthro...",None,"사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 ...",7,1.631312,19993858-dd49-44a9-b7d3-529a442eb39a,3a0ce64e-cb80-419b-83e0-6e867168d0e4
1,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 ‘삼성 가우스’입니다.,None,삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.,5,1.088572,f7d94096-7bcb-439b-832e-96d2dbd27169,29c4cd5b-ec4f-447a-899b-b61162588d4b
2,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 ‘삼성 가우스’입니다.,None,삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.,6,1.008772,2c20a8eb-ea28-4a83-8535-1f602cdcfc57,da1cc65b-be93-4f73-a931-bcc355560720
3,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,코히어(Cohere)의 데이터 출처 탐색기(Data Provenance Explor...,None,코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스...,9,3.309596,2e043f96-3f75-4a52-b65b-dbad66f4e58f,a74a1991-65db-44cd-bdc8-bd313145a4f2
4,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,None,2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.,2,1.025025,6c99b3e8-92f9-460f-9861-08699eaf317b,c04c3a57-dbe9-47cb-8691-42f5f3331dd6


In [99]:
# Context 를 반환하는 RAG 결과 반환 함수
def context_answer_rag_answer(inputs: dict):
    context = retriever.invoke(inputs["question"])
    return {
        "context": "\n".join([doc.page_content for doc in context]),
        "answer": chain.invoke(inputs["question"]),
        "question": inputs["question"],
    }


In [100]:
from langchain import hub

# 평가자 Prompt 가져오기
llm_evaluator_prompt = hub.pull("teddynote/context-answer-evaluator")
llm_evaluator_prompt.pretty_print()



As an LLM evaluator (judge), please assess the LLM's response to the given question. Evaluate the response's accuracy, comprehensiveness, and context precision based on the provided context. After your evaluation, return only the numerical scores in the following format:
Accuracy: [score]
Comprehensiveness: [score]
Context Precision: [score]
Final: [normalized score]
Grading rubric:

Accuracy (0-10 points):
Evaluate how well the answer aligns with the information provided in the given context.

0 points: The answer is completely inaccurate or contradicts the provided context
4 points: The answer partially aligns with the context but contains significant inaccuracies
7 points: The answer mostly aligns with the context but has minor inaccuracies or omissions
10 points: The answer fully aligns with the provided context and is completely accurate


Comprehensiveness (0-10 points):

0 points: The answer is completely inadequate or irrelevant
3 points: The answer is accurate but too brief t

In [101]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 평가자 생성
custom_llm_evaluator = (
    llm_evaluator_prompt
    | ChatOpenAI(temperature=0.0, model="gpt-4.1")
    | StrOutputParser()
)


In [102]:
# 답변을 생성합니다.
output = context_answer_rag_answer(
    {"question": "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?"}
)

# 점수 평가 실행
custom_llm_evaluator.invoke(output)


'1.0'

In [103]:
from langsmith.schemas import Run, Example


def custom_evaluator(run: Run, example: Example) -> dict:
    # LLM 생성 답변, 정답 답변 가져오기
    llm_answer = run.outputs.get("answer", "")
    context = run.outputs.get("context", "")
    question = example.outputs.get("question", "")

    # 랜덤 점수 반환
    score = custom_llm_evaluator.invoke(
        {"question": question, "answer": llm_answer, "context": context}
    )
    return {"key": "custom_score", "score": float(score)}


In [104]:
from langsmith.evaluation import evaluate

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 실행
experiment_results = evaluate(
    context_answer_rag_answer,
    data=dataset_name,
    evaluators=[custom_evaluator],
    experiment_prefix="CUSTOM-LLM-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Custom LLM Evaluator 활용한 평가",
    },
)
experiment_results

View the evaluation results for experiment: 'CUSTOM-LLM-EVAL-e8d5beb1' at:
https://smith.langchain.com/o/53db61eb-d386-4c02-9de8-5b28476b0edc/datasets/70f9dcf3-893a-4e50-80a9-35318578ee8c/compare?selectedSessions=d8b61bca-bcbd-4d64-8d72-e4b460e5a7c6




0it [00:00, ?it/s]

Error running evaluator <DynamicRunEvaluator custom_evaluator> on run aa0769e5-caca-4eb6-972a-5164f179dd5a: ValueError("could not convert string to float: 'Accuracy: 10\\nComprehensiveness: 10\\nContext Precision: 10\\nFinal: 1.0'")
Traceback (most recent call last):
  File "/opt/anaconda3/envs/LangChain/lib/python3.11/site-packages/langsmith/evaluation/_runner.py", line 1620, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(  # type: ignore[call-arg]
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/LangChain/lib/python3.11/site-packages/langsmith/evaluation/evaluator.py", line 351, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "/var/folders/sj/3r49bs6178v7nfhcxh7lpzd80000gn/T/ipykernel_57383/3052928486.py", line 14, in custom_evaluator
    return {"key": "custom_score", "score": float(score)}
                                            ^^^^^^^^^^^^
ValueError: could not convert string to f

,inputs.question,outputs.context,outputs.answer,outputs.question,error,reference.answer,feedback.custom_score,execution_time,example_id,id
0,구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,"£ 구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공\nn 구...","아니요, 구글이 20억 달러를 투자한 기업은 테디노트가 아니라 앤스로픽(Anthro...",구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,None,"사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 ...",NaN,2.822974,19993858-dd49-44a9-b7d3-529a442eb39a,aa0769e5-caca-4eb6-972a-5164f179dd5a
1,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,"▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··············...",삼성전자가 만든 생성형 AI의 이름은 ‘삼성 가우스’입니다.,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,None,삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.,1.00,1.816383,f7d94096-7bcb-439b-832e-96d2dbd27169,8ae902c4-66f4-421d-b7a9-9c40b2dd4384
2,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,"▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··············...",삼성전자가 만든 생성형 AI의 이름은 ‘삼성 가우스’입니다.,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,None,삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.,1.00,1.807365,2c20a8eb-ea28-4a83-8535-1f602cdcfc57,f1b1780e-42b9-4cf2-80b7-c6ba675ffce2
3,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,"SPRi AI Brief | \n2023-12월호\n8\n코히어, 데이터 투명성 ...",코히어(Cohere)의 데이터 출처 탐색기(Data Provenance Explor...,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,None,코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스...,0.97,3.563904,2e043f96-3f75-4a52-b65b-dbad66f4e58f,76cb6924-41eb-446d-94cb-159124620778
4,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,1. 정책/법제 \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미...,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,None,2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.,1.00,1.627517,6c99b3e8-92f9-460f-9861-08699eaf317b,761bbc7d-9ee7-4183-8587-1ca9330afab7


08. Rouge, BLEU, METEOR, SemScore 기반 휴리스틱 평가

In [105]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI

# PDFRAG 객체 생성
rag = PDFRAG(
    "data/SPRI_AI_Brief_2023년12월호_F.pdf",
    ChatOpenAI(model="gpt-4.1", temperature=0),
)

# 검색기(retriever) 생성
retriever = rag.create_retriever()

# 체인(chain) 생성
chain = rag.create_chain(retriever)

# 질문에 대한 답변 생성
chain.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")


'삼성전자가 자체 개발한 생성형 AI의 이름은 ‘삼성 가우스’입니다.'

In [4]:
# 질문에 대한 답변하는 함수를 생성
def ask_question(inputs: dict):
    return {"answer": chain.invoke(inputs["question"])}


In [107]:
from langchain_teddynote.community.kiwi_tokenizer import KiwiTokenizer

# 토크나이저 선언
kiwi_tokenizer = KiwiTokenizer()

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"

# 토큰화
print(sent1.split())
print(sent2.split())

print("===" * 20)

# 토큰화
print(kiwi_tokenizer.tokenize(sent1))
print(kiwi_tokenizer.tokenize(sent2))


['안녕하세요.', '반갑습니다.', '내', '이름은', '테디입니다.']
['안녕하세용', '반갑습니다~^^', '내', '이름은', '테디입니다!!']
['안녕', '하', '세요', '.', '반갑', '습니다', '.', '나', '의', '이름', '은', '테디', '이', 'ᆸ니다', '.']
['안녕', '하', '세요', 'ᆼ', '반갑', '습니다', '~', '^^', '나', '의', '이름', '은', '테디', '이', 'ᆸ니다', '!!']


In [108]:
from rouge_score import rouge_scorer

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"
sent3 = "내 이름은 테디입니다. 안녕하세요. 반갑습니다."

scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeL"], use_stemmer=False, tokenizer=KiwiTokenizer()
)

print(
    f"[1] {sent1}\n[2] {sent2}\n[rouge1] {scorer.score(sent1, sent2)['rouge1'].fmeasure:.5f}\n[rouge2] {scorer.score(sent1, sent2)['rouge2'].fmeasure:.5f}\n[rougeL] {scorer.score(sent1, sent2)['rougeL'].fmeasure:.5f}"
)
print("===" * 20)
print(
    f"[1] {sent1}\n[2] {sent3}\n[rouge1] {scorer.score(sent1, sent3)['rouge1'].fmeasure:.5f}\n[rouge2] {scorer.score(sent1, sent3)['rouge2'].fmeasure:.5f}\n[rougeL] {scorer.score(sent1, sent3)['rougeL'].fmeasure:.5f}"
)


[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!
[rouge1] 0.77419
[rouge2] 0.62069
[rougeL] 0.77419
[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 내 이름은 테디입니다. 안녕하세요. 반갑습니다.
[rouge1] 1.00000
[rouge2] 0.92857
[rougeL] 0.53333


In [109]:
from nltk.translate.bleu_score import sentence_bleu

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"
sent3 = "내 이름은 테디입니다. 안녕하세요. 반갑습니다."

# 토큰화
print(kiwi_tokenizer.tokenize(sent1, type="sentence"))
print(kiwi_tokenizer.tokenize(sent2, type="sentence"))
print(kiwi_tokenizer.tokenize(sent3, type="sentence"))


안녕 하 세요 . 반갑 습니다 . 나 의 이름 은 테디 이 ᆸ니다 .
안녕 하 세요 ᆼ 반갑 습니다 ~ ^^ 나 의 이름 은 테디 이 ᆸ니다 !!
나 의 이름 은 테디 이 ᆸ니다 . 안녕 하 세요 . 반갑 습니다 .


In [110]:
bleu_score = sentence_bleu(
    [kiwi_tokenizer.tokenize(sent1, type="sentence")],
    kiwi_tokenizer.tokenize(sent2, type="sentence"),
)
print(f"[1] {sent1}\n[2] {sent2}\n[score] {bleu_score:.5f}")
print("===" * 20)

bleu_score = sentence_bleu(
    [kiwi_tokenizer.tokenize(sent1, type="sentence")],
    kiwi_tokenizer.tokenize(sent3, type="sentence"),
)
print(f"[1] {sent1}\n[2] {sent3}\n[score] {bleu_score:.5f}")


[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!
[score] 0.74879
[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 내 이름은 테디입니다. 안녕하세요. 반갑습니다.
[score] 0.95739


In [113]:
from nltk.corpus import wordnet as wn
import nltk

nltk.download('wordnet')

wn.ensure_loaded()


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mypsyche/nltk_data...


In [114]:
from nltk.translate import meteor_score

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"
sent3 = "내 이름은 테디입니다. 안녕하세요. 반갑습니다."

meteor = meteor_score.meteor_score(
    [kiwi_tokenizer.tokenize(sent1, type="list")],
    kiwi_tokenizer.tokenize(sent2, type="list"),
)

print(f"[1] {sent1}\n[2] {sent2}\n[score] {meteor:.5f}")
print("===" * 20)

meteor = meteor_score.meteor_score(
    [kiwi_tokenizer.tokenize(sent1, type="list")],
    kiwi_tokenizer.tokenize(sent3, type="list"),
)
print(f"[1] {sent1}\n[2] {sent3}\n[score] {meteor:.5f}")


[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!
[score] 0.78849
[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 내 이름은 테디입니다. 안녕하세요. 반갑습니다.
[score] 0.96800


In [115]:
from sentence_transformers import SentenceTransformer, util
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"
sent3 = "내 이름은 테디입니다. 안녕하세요. 반갑습니다."

# SentenceTransformer 모델 로드
model = SentenceTransformer("all-mpnet-base-v2")

# 문장들을 인코딩
sent1_encoded = model.encode(sent1, convert_to_tensor=True)
sent2_encoded = model.encode(sent2, convert_to_tensor=True)
sent3_encoded = model.encode(sent3, convert_to_tensor=True)

# sent1과 sent2 사이의 코사인 유사도 계산
cosine_similarity = util.pytorch_cos_sim(sent1_encoded, sent2_encoded).item()
print(f"[1] {sent1}\n[2] {sent2}\n[score] {cosine_similarity:.5f}")

print("===" * 20)

# sent1과 sent3 사이의 코사인 유사도 계산
cosine_similarity = util.pytorch_cos_sim(sent1_encoded, sent3_encoded).item()
print(f"[1] {sent1}\n[2] {sent3}\n[score] {cosine_similarity:.5f}")


README.md: 0.00B [00:00, ?B/s]

: 

: 

: 

In [1]:
from langsmith.schemas import Run, Example
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate import meteor_score
from sentence_transformers import SentenceTransformer, util
import os

# 토크나이저 병렬화 설정(HuggingFace 모델 사용)
os.environ["TOKENIZERS_PARALLELISM"] = "true"


def rouge_evaluator(metric: str = "rouge1") -> dict:
    # wrapper function 정의
    def _rouge_evaluator(run: Run, example: Example) -> dict:
        # 출력값과 정답 가져오기
        student_answer = run.outputs.get("answer", "")
        reference_answer = example.outputs.get("answer", "")

        # ROUGE 점수 계산
        scorer = rouge_scorer.RougeScorer(
            ["rouge1", "rouge2", "rougeL"], use_stemmer=True, tokenizer=KiwiTokenizer()
        )
        scores = scorer.score(reference_answer, student_answer)

        # ROUGE 점수 반환
        rouge = scores[metric].fmeasure

        return {"key": "ROUGE", "score": rouge}

    return _rouge_evaluator


def bleu_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # 토큰화
    reference_tokens = kiwi_tokenizer.tokenize(reference_answer, type="sentence")
    student_tokens = kiwi_tokenizer.tokenize(student_answer, type="sentence")

    # BLEU 점수 계산
    bleu_score = sentence_bleu([reference_tokens], student_tokens)

    return {"key": "BLEU", "score": bleu_score}


def meteor_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # 토큰화
    reference_tokens = kiwi_tokenizer.tokenize(reference_answer, type="list")
    student_tokens = kiwi_tokenizer.tokenize(student_answer, type="list")

    # METEOR 점수 계산
    meteor = meteor_score.meteor_score([reference_tokens], student_tokens)

    return {"key": "METEOR", "score": meteor}


def semscore_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # SentenceTransformer 모델 로드
    model = SentenceTransformer("all-mpnet-base-v2")

    # 문장 임베딩 생성
    student_embedding = model.encode(student_answer, convert_to_tensor=True)
    reference_embedding = model.encode(reference_answer, convert_to_tensor=True)

    # 코사인 유사도 계산
    cosine_similarity = util.pytorch_cos_sim(
        student_embedding, reference_embedding
    ).item()

    return {"key": "sem_score", "score": cosine_similarity}


In [5]:
from langsmith.evaluation import evaluate

# 평가자 정의
heuristic_evalulators = [
    rouge_evaluator(metric="rougeL"),
    bleu_evaluator,
    meteor_evaluator,
    semscore_evaluator,
]

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 실험 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=heuristic_evalulators,
    experiment_prefix="Heuristic-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Heuristic-EVAL (Rouge, BLEU, METEOR, SemScore) 을 사용하여 평가",
    },
)
experiment_results

/opt/anaconda3/envs/LangChain/lib/python3.11/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


LangSmithAuthError: Authentication failed for /datasets. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/datasets?limit=1&name=RAG_EVAL_DATASET', '{"detail":"Invalid token"}')